In [ ]:
# Import necessary libraries
import nest  # NEST simulator for spiking neural networks
import matplotlib.pyplot as plt  # For plotting results
import ipywidgets as widgets  # For interactive sliders
import numpy as np
from ipywidgets import interact, FloatSlider, IntSlider

# Set NEST verbosity to suppress extra NEST logs
nest.set_verbosity("M_WARNING")

In [ ]:
# Print default parameters of a neuron model
defaults = nest.GetDefaults("iaf_psc_alpha")
default_synapse = nest.GetDefaults("static_synapse")

print("Model Defaults")
for key, value in defaults.items():
    print(f"{key}: {value}")

print("Synaptic Defaults")
for key, value in default_synapse.items():
    print(f"{key}: {value}")

Create a single neuron (iaf_psc_alpha in NEST)

In [ ]:
def run_one_neuron(I_e, sim_time, C_m, tau_m, V_reset, V_th, t_ref):
    # Reset the NEST kernel (to clear previous simulations)
    nest.ResetKernel()

    # Set the simulation resolution
    nest.SetKernelStatus({"resolution": 0.01})
    
    # Create a single IAF neuron
    neuron = nest.Create('iaf_psc_alpha')

    # Set neuron parameters
    nest.SetStatus(neuron, {
        "I_e": I_e,              # Input current (pA)
        "C_m": C_m,              # Membrane capacitance (pF)
        "tau_m": tau_m,          # Membrane time constant (ms)
        "V_reset": V_reset,      # Reset potential (mV)
        "V_th": V_th,             # Threshold potential (mV)
        "t_ref": t_ref         # Refractory period (ms)
    })

    # Create devices for recording and stimulation
    spike_recorder = nest.Create('spike_recorder')  # Record spikes
    multimeter = nest.Create('multimeter')         # Record membrane potential

    # Configure the multimeter to record the membrane potential
    nest.SetStatus(multimeter, {"record_from": ["V_m"]})
    nest.SetStatus(multimeter, {"interval": 0.01})

    # Connect the multimeter and spike recorder to the neuron
    nest.Connect(multimeter, neuron)
    nest.Connect(neuron, spike_recorder)

    # Simulate for the specified time
    nest.Simulate(sim_time)

    # Retrieve and plot the membrane potential
    events = nest.GetStatus(multimeter, 'events')[0]  # Get recorded events
    time = events['times']                            # Time data
    V_m = events['V_m']                               # Membrane potential data

    # Retrieve spike events
    spikes = nest.GetStatus(spike_recorder, 'events')[0]  # Get spike events
    spike_times = spikes['times']                        # Extract spike times

    # Print spike times
    print(f'Spike Times: {", ".join(map(str, spike_times))}')

    # Calculate firing rate
    firing_rate = len(spike_times) / (sim_time / 1000.0)  # Spikes per second (Hz)
    print(f'Firing Rate: {firing_rate:.2f} Hz')

    # Plot the membrane potential over time
    plt.figure(figsize=(10, 6))
    plt.plot(time, V_m, label='Membrane Potential (V_m)')
    plt.xlabel('Time (ms)')
    plt.ylabel('Membrane Potential (mV)')
    plt.title('IAF Neuron Membrane Potential Over Time')
    plt.axhline(-55, color='gray', linestyle='--', label='Threshold (V_th)')
    plt.legend()
    plt.grid()
    plt.show()

# Interactive widgets for input parameters
I_e = widgets.FloatSlider(value=187.52, min=0.0, max=100000.0, step=0.01, description='I_e (pA):')
sim_time = widgets.FloatSlider(value=10000.0, min=1.0, max=60000.0, step=1.0, description='Sim Time (ms):')
C_m = widgets.FloatSlider(value=250.0, min=1.0, max=1000.0, step=1.0, description='C_m (pF):')
tau_m = widgets.FloatSlider(value=20.0, min=1.0, max=100.0, step=0.1, description='tau_m (ms):')
V_reset = widgets.FloatSlider(value=-70.0, min=-100.0, max=0.0, step=1.0, description='V_reset (mV):')
V_th = widgets.FloatSlider(value=-55.0, min=-100.0, max=0.0, step=1.0, description='V_th (mV):')
t_ref = widgets.FloatSlider(value=8.0, min=0.0, max=100.0, step=1.0, description='t_ref (ms):')
widgets.interact(run_one_neuron, 
I_e=I_e, sim_time=sim_time, C_m=C_m, tau_m=tau_m, V_reset=V_reset, V_th=V_th, t_ref=t_ref
)


Simulates a single self-connected neuron.

In [ ]:
def one_neuron_self_conn(I_e, sim_time, C_m, tau_m, V_reset, V_th, t_ref, weight):
    # Reset the NEST kernel (to clear previous simulations)
    nest.ResetKernel()

    # Set the simulation resolution
    nest.SetKernelStatus({"resolution": 0.01})

    # Create a single IAF neuron
    neuron = nest.Create('iaf_psc_alpha')

    # Set neuron parameters
    nest.SetStatus(neuron, {
        "I_e": I_e,              # Input current (pA)
        "C_m": C_m,              # Membrane capacitance (pF)
        "tau_m": tau_m,          # Membrane time constant (ms)
        "V_reset": V_reset,      # Reset potential (mV)
        "V_th": V_th,             # Threshold potential (mV)
        "t_ref": t_ref         # Refractory period (ms)
    })

    # Create devices for recording and stimulation
    spike_recorder = nest.Create('spike_recorder')  # Record spikes
    multimeter = nest.Create('multimeter')         # Record membrane potential

    # Configure the multimeter to record the membrane potential
    nest.SetStatus(multimeter, {"record_from": ["V_m"]})
    nest.SetStatus(multimeter, {"interval": 0.01})

    # Connect the multimeter and spike recorder to the neuron
    nest.Connect(multimeter, neuron)
    nest.Connect(neuron, spike_recorder)

    # Self-connect the neuron with specified weight
    nest.Connect(neuron, neuron, syn_spec={"weight": weight})

    # Simulate for the specified time
    nest.Simulate(sim_time)

    # Retrieve and plot the membrane potential
    events = nest.GetStatus(multimeter, 'events')[0]  # Get recorded events
    time = events['times']                            # Time data
    V_m = events['V_m']                               # Membrane potential data

    # Retrieve spike events
    spikes = nest.GetStatus(spike_recorder, 'events')[0]  # Get spike events
    spike_times = spikes['times']                        # Extract spike times

    # Print spike times
    print(f'Spike Times: {", ".join(map(str, spike_times))}')

    # Calculate firing rate
    firing_rate = len(spike_times) / (sim_time / 1000.0)  # Spikes per second (Hz)
    print(f'Firing Rate: {firing_rate:.2f} Hz')

    # Plot the membrane potential over time
    plt.figure(figsize=(10, 6))
    plt.plot(time, V_m, label='Membrane Potential (V_m)')
    plt.xlabel('Time (ms)')
    plt.ylabel('Membrane Potential (mV)')
    plt.title('IAF Neuron Membrane Potential Over Time')
    plt.axhline(-55, color='gray', linestyle='--', label='Threshold (V_th)')
    plt.legend()
    plt.grid()
    plt.show()

# Interactive widgets for input parameters
I_e = widgets.FloatSlider(value=187.52 , min=0.0, max=1000.0, step=0.01, description='I_e (pA):')
sim_time = widgets.FloatSlider(value=10000.0, min=1.0, max=60000.0, step=1.0, description='Sim Time (ms):')
C_m = widgets.FloatSlider(value=250.0, min=1.0, max=1000.0, step=1.0, description='C_m (pF):')
tau_m = widgets.FloatSlider(value=20.0, min=1.0, max=100.0, step=0.1, description='tau_m (ms):')
V_reset = widgets.FloatSlider(value=-70.0, min=-100.0, max=0.0, step=1.0, description='V_reset (mV):')
V_th = widgets.FloatSlider(value=-55.0, min=-100.0, max=0.0, step=1.0, description='V_th (mV):')
t_ref = widgets.FloatSlider(value=8.0, min=0.0, max=100.0, step=1.0, description='t_ref (ms):')
weight = widgets.FloatSlider(value=4511.0, min=-1.0, max=10000.0, step=0.1, description='Weight:')

widgets.interact(one_neuron_self_conn, 
I_e=I_e, sim_time=sim_time, C_m=C_m, tau_m=tau_m, V_reset=V_reset, V_th=V_th, t_ref=t_ref, weight=weight
)
